In [1]:
import os
os.chdir('../streamlit')


In [9]:
from utils import get_round_data, read_file, ROUND_INFO_CSV

# In your page that needs Area data
round_data = get_round_data(ex_50=True, ex_incomplete=False)
# round_data

round_info = read_file(ROUND_INFO_CSV)

course_areas = round_info[['Course', 'Area']].drop_duplicates().set_index('Course')['Area']
round_data['Area'] = round_data['Course'].map(course_areas)

# Now you have Area column
# print(round_data[['Course', 'Area']])
round_data

,TEG,Round,Date,TEGNum,Player,Course,Pl,TEG-Round,Year,HC,Sc,GrossVP,NetVP,Stableford,Area
0,TEG 10,1,30/09/2017,10,Alex BAKER,Boavista,AB,TEG 10|R1,2017,32.0,99.0,28.0,-4.0,41.0,"Algarve, Portugal"
1,TEG 10,1,30/09/2017,10,David MULLIN,Boavista,DM,TEG 10|R1,2017,17.0,75.0,4.0,-13.0,49.0,"Algarve, Portugal"
2,TEG 10,1,30/09/2017,10,Gregg WILLIAMS,Boavista,GW,TEG 10|R1,2017,26.0,96.0,25.0,-1.0,38.0,"Algarve, Portugal"
3,TEG 10,1,30/09/2017,10,John PATTERSON,Boavista,JP,TEG 10|R1,2017,21.0,102.0,31.0,10.0,29.0,"Algarve, Portugal"
4,TEG 10,1,30/09/2017,10,Jon BAKER,Boavista,JB,TEG 10|R1,2017,19.0,89.0,18.0,-1.0,37.0,"Algarve, Portugal"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,TEG 9,4,03/10/2016,9,David MULLIN,Praia D'El Rey,DM,TEG 9|R4,2016,17.0,89.0,16.0,-1.0,38.0,"Lisbon Coast, Portugal"
331,TEG 9,4,03/10/2016,9,Gregg WILLIAMS,Praia D'El Rey,GW,TEG 9|R4,2016,26.0,100.0,27.0,1.0,38.0,"Lisbon Coast, Portugal"
332,TEG 9,4,03/10/2016,9,John PATTERSON,Praia D'El Rey,JP,TEG 9|R4,2016,27.0,90.0,17.0,-10.0,47.0,"Lisbon Coast, Portugal"
333,TEG 9,4,03/10/2016,9,Jon BAKER,Praia D'El Rey,JB,TEG 9|R4,2016,19.0,92.0,19.0,0.0,36.0,"Lisbon Coast, Portugal"


In [7]:
from utils import load_all_data

all_data = load_all_data(exclude_teg_50 = True, exclude_incomplete_tegs = True)

max_stab = max(all_data['Stableford'])

# stab_dist = all_data['Stableford'].value_counts().reset_index()
# stab_dist.columns = ['Stableford','Count']
# print(stab_dist)

all_data['Stableford'].value_counts().reset_index().sort_values(by='Stableford')

,Stableford,count
3,0.0,756
2,1.0,1153
0,2.0,2044
1,3.0,1707
4,4.0,340
5,5.0,30
